In [1]:
import tensorflow as tf

In [2]:
!nvidia-smi

Fri Aug  5 17:12:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   38C    P0    86W / 260W |    343MiB / 11019MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
#sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
# sys.path.insert(0,'/home/jk21/Documents/hmd/jk_classifier/lucashnegri-peakutils-51a679cd8428')
# sys.path.insert(0,'/home/jk21/Documents/hmd/jk_classifier/S1-S1-Phonocardiogram-Peak-Detection-Method-in-Python')
sys.path.insert(0,'utils')
from helper_code import *
from get_feature import *
from models import *
from Generator0 import *
from keras.preprocessing import sequence
# import peakutils
from scipy import special
import scipy.io as sio
from keras.preprocessing.sequence import pad_sequences
sys.path.insert(0,'/home/jk21/Documents/hmd/jk_classifier/lucashnegri-peakutils-51a679cd8428')
import peakutils

2022-08-05 17:12:41.034563: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-05 17:12:41.043141: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-05 17:12:41.043843: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-05 17:12:41.045208: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
pwd

'/home/jk21/Documents/hmd/jk_classifier'

In [5]:
root_dir = 'physionet.org/files/circor-heart-sound/1.0.3'
# training_data_file = root_dir + '/' + 'training_data.csv'
# training_data_dir = root_dir + '/' + 'training_data'
model_dir = root_dir + '/' + 'model'

In [6]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Fri Aug  5 11:31:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   39C    P2    82W / 260W |  10485MiB / 11019MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
pwd

'/home/jk21/Documents/hmd/jk_classifier'

In [8]:
# data_folder =  'physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/home/jk21/Downloads/Data/data/murmur/train'
test_folder = '/home/jk21/Downloads/Data/data/murmur/test'


In [9]:
model_folder = 'lcnn2'

In [10]:
model_folder

'lcnn2'

In [11]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint

In [12]:
def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
    if e < start:
        return lr_start
    elif e > end:
        return lr_end

    middle = (start + end) / 2
    s = lambda x: 1 / (1 + np.exp(-x))

    return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

In [13]:
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

In [14]:
params_feature = {'samp_sec': 30,
                  #### melspec, stft 피쳐 옵션들  
                  'pre_emphasis': 0,
                  'hop_length': 128,
                  'win_length':256,
                  'n_mels': 100,
                  #### cqt 피쳐 옵션들  
                  'filter_scale': 1,
                  'n_bins': 80,
                  'fmin': 10,
                  'maxlen': 60,
                  'maxlen1': 120000,
                  'min_dist':500,
}

In [28]:
features_trn,mel_input_shape, cqt_input_shape, stft_input_shape = get_features_3lb_all(train_folder, patient_files_trn, **params_feature,use_mel = True, use_cqt = False, use_stft = False,use_raw=False,use_interval=True,use_wav2=True)

 22%|█████████                                | 167/751 [09:56<38:15,  3.93s/it]

/home/jk21/Downloads/Data/data/murmur/train/50216_Phc_1.wav


 29%|███████████▉                             | 218/751 [13:41<43:27,  4.89s/it]

/home/jk21/Downloads/Data/data/murmur/train/50321_MV_1.wav


 47%|███████████████████▎                     | 354/751 [25:03<37:20,  5.64s/it]

/home/jk21/Downloads/Data/data/murmur/train/68269_PV.wav


100%|███████████████████████████████████████| 751/751 [1:05:38<00:00,  5.24s/it]


melspec:  100 938
cqt:  1 1
stft:  1 1


100%|███████████████████████████████████████| 2532/2532 [03:35<00:00, 11.78it/s]


In [ ]:
from numba import cuda

cuda.select_device(0)

cuda.close()

In [29]:
features_test,mel_input_shape, cqt_input_shape, stft_input_shape= get_features_3lb_all(test_folder, patient_files_test, **params_feature,use_mel = True, use_cqt = False, use_stft = False,use_raw=False,use_interval=True,use_wav2=True)

 27%|███████████▏                              | 51/191 [04:21<10:13,  4.38s/it]

/home/jk21/Downloads/Data/data/murmur/test/50348_AV_1.wav


100%|█████████████████████████████████████████| 191/191 [17:45<00:00,  5.58s/it]


melspec:  100 938
cqt:  1 1
stft:  1 1


100%|█████████████████████████████████████████| 631/631 [01:07<00:00,  9.40it/s]


In [17]:
feature_info = {'mel' : True,'stft': False,'cqt': False,'interval':True,'maxlen': 80, 'min_dist':500}

In [30]:
with open('/home/jk21/Downloads/Data/features_trn_wav2.pickle', 'wb') as f:
    pickle.dump(features_trn, f, pickle.HIGHEST_PROTOCOL)

In [31]:
with open('/home/jk21/Downloads/Data/features_test_wav2.pickle', 'wb') as f:
    pickle.dump(features_test, f, pickle.HIGHEST_PROTOCOL)

In [15]:
with open('/home/jk21/Downloads/Data/features_trn_wav2.pickle', 'rb') as f:
    features_trn = pickle.load(f)

In [16]:
with open('/home/jk21/Downloads/Data/features_test_wav2.pickle', 'rb') as f:
    features_test = pickle.load(f)

In [17]:
mel_input_shape = features_trn['mel1'].shape[1:]

In [18]:
mel_input_shape

(100, 938, 1)

In [19]:
cqt_input_shape = features_trn['cqt1'].shape[1:]

In [20]:
cqt_input_shape

(1, 1, 1)

In [21]:
stft_input_shape = features_trn['stft1'].shape[1:]

In [22]:
stft_input_shape

(1, 1, 1)

In [23]:
interval_input_shape= features_trn['interval'].shape[1:]

In [24]:
interval_input_shape

(60, 1)

In [25]:
wav2_input_shape= features_trn['wav2'].shape[1:]

In [26]:
wav2_input_shape

(374, 32)

In [27]:
features_test['raw1']

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [28]:
for i in range(len(features_trn['interval'])):
    features_trn['interval'][i]=np.array(features_trn['interval'][i])

In [29]:
for i in range(len(features_trn['mel1'])):
    features_trn['mel1'][i]=np.array(features_trn['mel1'][i])

In [30]:
model1 = get_LCNN_o_4_dr(mel_input_shape, cqt_input_shape,stft_input_shape, interval_input_shape, wav2_input_shape,use_mel=True, use_stft = False,use_cqt=False,ext = True,ord1=False)

In [31]:

model2 = get_LCNN_o_4_dr(mel_input_shape, cqt_input_shape,stft_input_shape, interval_input_shape, wav2_input_shape,use_mel=True, use_stft = False,use_cqt=False,ext = True,ord1=True)


In [31]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mel (InputLayer)               [(None, 100, 938, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 100, 938, 32  832         ['mel[0][0]']                    
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 100, 938, 32  832         ['mel[0][0]']                    
                                )                                                             

                                                                                                  
 conv2d_15 (Conv2D)             (None, 13, 118, 32)  1056        ['batch_normalization_8[0][0]']  
                                                                                                  
 conv1d (Conv1D)                (None, 58, 10)       40          ['interval[0][0]']               
                                                                                                  
 conv1d_2 (Conv1D)              (None, 372, 10)      970         ['wav2[0][0]']                   
                                                                                                  
 maximum_7 (Maximum)            (None, 13, 118, 32)  0           ['conv2d_14[0][0]',              
                                                                  'conv2d_15[0][0]']              
                                                                                                  
 batch_nor

 stft (InputLayer)              [(None, 1, 1, 1)]    0           []                               
                                                                                                  
 cqt (InputLayer)               [(None, 1, 1, 1)]    0           []                               
                                                                                                  
 dense_7 (Dense)                (None, 3)            129         ['concatenate_1[0][0]']          
                                                                                                  
Total params: 149,005
Trainable params: 146,781
Non-trainable params: 2,224
__________________________________________________________________________________________________


In [32]:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mel (InputLayer)               [(None, 100, 938, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_18 (Conv2D)             (None, 100, 938, 32  832         ['mel[0][0]']                    
                                )                                                                 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 100, 938, 32  832         ['mel[0][0]']                    
                                )                                                           

                                                                                                  
 conv2d_33 (Conv2D)             (None, 13, 118, 32)  1056        ['batch_normalization_18[0][0]'] 
                                                                                                  
 conv1d_4 (Conv1D)              (None, 58, 10)       40          ['interval[0][0]']               
                                                                                                  
 conv1d_6 (Conv1D)              (None, 372, 10)      970         ['wav2[0][0]']                   
                                                                                                  
 maximum_16 (Maximum)           (None, 13, 118, 32)  0           ['conv2d_32[0][0]',              
                                                                  'conv2d_33[0][0]']              
                                                                                                  
 batch_nor

 stft (InputLayer)              [(None, 1, 1, 1)]    0           []                               
                                                                                                  
 cqt (InputLayer)               [(None, 1, 1, 1)]    0           []                               
                                                                                                  
 dense_15 (Dense)               (None, 2)            86          ['concatenate_3[0][0]']          
                                                                                                  
Total params: 148,962
Trainable params: 146,738
Non-trainable params: 2,224
__________________________________________________________________________________________________


In [33]:
n_epoch = 50
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))


batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
          'lowpass': [.5, [11,12,13,14,15,16,17,18]],
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
#           'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'],
            features_trn['interval'],features_trn['wav2'],features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
           features_trn['mm_labels'],  ## our Y
                        **params)()


ValidDGen_1 = Generator0([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'],features_test['interval'],
                              features_test['wav2'], features_test['mel1'],features_test['cqt1'],features_test['stft1']], 
                             features_test['mm_labels'],  ## our Y
                        **params)()

class_weight = {0: 3, 1: 1., 2: 1} 

 
model1.fit(TrainDGen_1, 
           validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'],features_test['interval'],
                              features_test['wav2'], features_test['mel1'],features_test['cqt1'],features_test['stft1']], 
                             features_test['mm_labels']),
           callbacks=[lr],
           steps_per_epoch=np.ceil(len(features_trn['mm_labels'])/batch_size), 
           validation_steps=np.ceil(len(features_test['mm_labels'])/batch_size)
           ,epochs = n_epoch)



Epoch 1/50
40/40 [==============================] - 46s 1s/step - loss: 4.6650 - accuracy: 0.3453 - auc: 0.4876 - val_loss: 1.4523 - val_accuracy: 0.7448 - val_auc: 0.8451 - lr: 9.9851e-04
Epoch 2/50
40/40 [==============================] - 40s 999ms/step - loss: 0.7977 - accuracy: 0.6867 - auc: 0.8323 - val_loss: 0.7231 - val_accuracy: 0.7448 - val_auc: 0.8620 - lr: 9.9807e-04
Epoch 3/50
40/40 [==============================] - 40s 1s/step - loss: 0.6341 - accuracy: 0.7574 - auc: 0.8602 - val_loss: 0.6096 - val_accuracy: 0.7448 - val_auc: 0.8511 - lr: 9.9750e-04
Epoch 4/50
40/40 [==============================] - 40s 997ms/step - loss: 0.6303 - accuracy: 0.7516 - auc: 0.8655 - val_loss: 0.6108 - val_accuracy: 0.7448 - val_auc: 0.8485 - lr: 9.9676e-04
Epoch 5/50
40/40 [==============================] - 40s 998ms/step - loss: 0.6063 - accuracy: 0.7594 - auc: 0.8672 - val_loss: 0.5918 - val_accuracy: 0.7480 - val_auc: 0.8591 - lr: 9.9581e-04
Epoch 6/50
40/40 [============================

Epoch 44/50
40/40 [==============================] - 40s 991ms/step - loss: 0.5328 - accuracy: 0.7750 - auc: 0.8959 - val_loss: 0.4776 - val_accuracy: 0.8177 - val_auc: 0.9053 - lr: 1.9102e-05
Epoch 45/50
40/40 [==============================] - 40s 1s/step - loss: 0.5289 - accuracy: 0.7840 - auc: 0.8897 - val_loss: 0.4755 - val_accuracy: 0.8177 - val_auc: 0.9058 - lr: 1.7033e-05
Epoch 46/50
40/40 [==============================] - 40s 1s/step - loss: 0.5313 - accuracy: 0.7812 - auc: 0.8967 - val_loss: 0.4752 - val_accuracy: 0.8177 - val_auc: 0.9054 - lr: 1.5431e-05
Epoch 47/50
40/40 [==============================] - 40s 1s/step - loss: 0.5173 - accuracy: 0.7945 - auc: 0.8943 - val_loss: 0.4740 - val_accuracy: 0.8130 - val_auc: 0.9061 - lr: 1.4193e-05
Epoch 48/50
40/40 [==============================] - 40s 991ms/step - loss: 0.5384 - accuracy: 0.7715 - auc: 0.8941 - val_loss: 0.4731 - val_accuracy: 0.8209 - val_auc: 0.9066 - lr: 1.3236e-05
Epoch 49/50
40/40 [=========================

In [32]:
n_epoch = 50
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))


batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
         'lowpass': [.5, [11,12,13,14,15,16,17,18]],
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
#           'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'],
            features_trn['interval'],features_trn['wav2'],features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
           features_trn['out_labels'],  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()
class_weight = {0: 3, 1: 1., 2: 1} 
    
model2.fit(TrainDGen_1,
           validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'],features_test['interval'],
                              features_test['wav2'], features_test['mel1'],features_test['cqt1'],features_test['stft1']], 
                             features_test['out_labels']),
           callbacks=[lr],
           steps_per_epoch=np.ceil(len(features_trn['out_labels'])/batch_size), 
           validation_steps=np.ceil(len(features_test['out_labels'])/batch_size),
           epochs = n_epoch)


Epoch 1/50
40/40 [==============================] - 47s 1s/step - loss: 0.8732 - accuracy: 0.5180 - auc: 0.5183 - val_loss: 6.9100 - val_accuracy: 0.5008 - val_auc: 0.5008 - lr: 9.9851e-04
Epoch 2/50
40/40 [==============================] - 40s 1s/step - loss: 0.7136 - accuracy: 0.5312 - auc: 0.5511 - val_loss: 1.4941 - val_accuracy: 0.5008 - val_auc: 0.5802 - lr: 9.9807e-04
Epoch 3/50
40/40 [==============================] - 40s 1s/step - loss: 0.7044 - accuracy: 0.5398 - auc: 0.5559 - val_loss: 0.6860 - val_accuracy: 0.5166 - val_auc: 0.5807 - lr: 9.9750e-04
Epoch 4/50
40/40 [==============================] - 40s 999ms/step - loss: 0.7012 - accuracy: 0.5406 - auc: 0.5569 - val_loss: 0.7007 - val_accuracy: 0.5198 - val_auc: 0.5775 - lr: 9.9676e-04
Epoch 5/50
40/40 [==============================] - 40s 1s/step - loss: 0.6952 - accuracy: 0.5355 - auc: 0.5669 - val_loss: 0.6670 - val_accuracy: 0.6133 - val_auc: 0.6544 - lr: 9.9581e-04
Epoch 6/50
40/40 [==============================] - 

Epoch 44/50
40/40 [==============================] - 40s 998ms/step - loss: 0.6745 - accuracy: 0.5742 - auc: 0.6412 - val_loss: 0.6361 - val_accuracy: 0.6593 - val_auc: 0.7210 - lr: 1.9102e-05
Epoch 45/50
40/40 [==============================] - 40s 1s/step - loss: 0.6786 - accuracy: 0.5789 - auc: 0.6287 - val_loss: 0.6366 - val_accuracy: 0.6624 - val_auc: 0.7204 - lr: 1.7033e-05
Epoch 46/50
40/40 [==============================] - 40s 998ms/step - loss: 0.6771 - accuracy: 0.5641 - auc: 0.6383 - val_loss: 0.6377 - val_accuracy: 0.6624 - val_auc: 0.7159 - lr: 1.5431e-05
Epoch 47/50
40/40 [==============================] - 40s 998ms/step - loss: 0.6755 - accuracy: 0.5848 - auc: 0.6417 - val_loss: 0.6374 - val_accuracy: 0.6593 - val_auc: 0.7170 - lr: 1.4193e-05
Epoch 48/50
40/40 [==============================] - 40s 997ms/step - loss: 0.6763 - accuracy: 0.5801 - auc: 0.6387 - val_loss: 0.6383 - val_accuracy: 0.6577 - val_auc: 0.7149 - lr: 1.3236e-05
Epoch 49/50
40/40 [===================

In [34]:
params_feature

{'samp_sec': 30,
 'pre_emphasis': 0,
 'hop_length': 128,
 'win_length': 256,
 'n_mels': 100,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'maxlen': 60,
 'maxlen1': 120000,
 'min_dist': 500}

In [35]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape
params_feature['interval_input_shape'] = interval_input_shape
params_feature['wav2_input_shape'] = wav2_input_shape

In [36]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, param_feature) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    param_feature['model1'] = m_name1
    param_feature['model2'] = m_name2
    param_feature['model_fnm1'] = filename1
    param_feature['model_fnm2'] = filename2

    with open(info_fnm, 'wb') as f:
        pk.dump(param_feature, f, pk.HIGHEST_PROTOCOL)
    return 1
    

In [37]:
model_folder

'lcnn2'

In [38]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'lcnn1', m_name2 = 'lcnn2', param_feature = params_feature)

1

In [39]:
def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)
#    if info_m['model'] == 'toy' :
#        model = get_toy(info_m['mel_shape'])
#    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
#    model.load_weights(filename)
    return info_m

In [40]:
model.summary()

Model: "wav2vec2-ctc"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 wav2vec2 (Wav2Vec2Model)    multiple                  94371712  
                                                                 
 dropout_26 (Dropout)        multiple                  0         
                                                                 
 lm_head (Dense)             multiple                  24608     
                                                                 
Total params: 94,396,320
Trainable params: 0
Non-trainable params: 94,396,320
_________________________________________________________________


In [63]:
def run_challenge_model(model, data, recordings, verbose,use_mel=True,use_stft=False,use_cqt=False,
                        maxlen=80,min_dist=500):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']
    
    if model['model1'] == 'lcnn1' :
        model1 = get_LCNN_o_4_dr(model['mel_shape'],model['cqt_shape'],model['stft_shape'], model['interval_input_shape'],model['wav2_input_shape'],use_mel = True, use_stft = False,use_cqt=False,ord1=False,ext=True)
    if model['model2'] == 'lcnn2' :
        model2 = get_LCNN_o_4_dr(model['mel_shape'],model['cqt_shape'],model['stft_shape'], model['interval_input_shape'], model['wav2_input_shape'],use_mel = True,use_stft = False,use_cqt=False,ord1=True,ext=True)
    model1.load_weights(model['model_fnm1'])
    model2.load_weights(model['model_fnm2'])
    
    maxlen = params_feature['maxlen']
    maxlen1 = params_feature['maxlen1']
    min_dist = params_feature['min_dist']
#    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose = 0)

    samp_sec = model['samp_sec'] 
    pre_emphasis = model['pre_emphasis']
    hop_length = model['hop_length']
    win_length = model['win_length']
    n_mels = model['n_mels']
    filter_scale = model['filter_scale']
    n_bins = model['n_bins']
    fmin = model['fmin']
    

    features['interval'] = []
    tmp_interval=[]  
    
    features['wav2'] = []
    tmp_raw = []
    tmp=[]      
    
    from wav2vec2 import Wav2Vec2ForCTC, Wav2Vec2Config
    import tensorflow as tf
    
    config = Wav2Vec2Config()
    tmp_model = Wav2Vec2ForCTC(config)
    tmp_model.trainable=False
    
    for i in range(len(recordings)):
        data = recordings[i]/32768
        tmp_raw.append(data)
        
    tmp_pad =pad_sequences(tmp_raw, maxlen=maxlen1, dtype='float64', padding='pre', truncating='pre', value=0.0)
    tmp_pad = tmp_pad[:,np.newaxis,:]
    
    for i in tqdm.tqdm(range(len(tmp_pad))):
        tmp_feature = tmp_model(tmp_pad[i])
        tmp.append(tmp_feature)
    
    tmp=np.array(tmp, dtype=np.float32)
    new_tmp1=tmp.reshape(-1,374,32)
    features['wav2']=new_tmp1
    
    
    
    for i in range(len(recordings)) :
        datos=recordings[i]
        filtros=sio.loadmat('/home/jk21/Documents/hmd/jk_classifier/S1-S1-Phonocardiogram-Peak-Detection-Method-in-Python/Filters1')
        try:
            X=datos
            X=X[12000:-12000]
            Fs=4000
            
            Fpa20=filtros['Fpa20'];			        # High pass filter
            Fpa20=Fpa20[0];					# High pass filter
            Fpb100=filtros['Fpb100'];		        # Low-pass Filter
            Fpb100=Fpb100[0];				# Low-pass Filter
            
            Xf=FpassBand(X,Fpa20,Fpb100); 	                # Apply a passband filter
            Xf=vec_nor(Xf);			
            
                    # Derivate of the Signal
            dX=derivate(Xf);				# Derivate of the signal
            dX=vec_nor(dX);					# Vector Normalizing
                    # Square of the signal
            dy=np.square(Xf);
            dy=vec_nor(dy);
                    
            size=np.shape(Xf)				# Rank or dimension of the array
            fil=size[0];					# Number of rows

            positive=np.zeros((1,fil+1));                   # Initializating Positives Values Vector 
            positive=positive[0];                           # Getting the Vector

            points=np.zeros((1,fil));                       # Initializating the all Peak Points Vector
            points=points[0];                               # Getting the point vector

            peaks=np.zeros((1,fil));                        # Initializating the s1-s1 Peak Vector
            peaks=peaks[0];                                 # Getting the point vector

            
            for i in range(0,fil):
                if dX[i]>0:
                    positive[i]=1;
                else:
                    positive[i]=0;

            for i in range(0,fil):
                if (positive[i]==1 and positive[i+1]==0):
                    points[i]=Xf[i];
                else:
                    points[i]=0;

            indexes=peakutils.indexes(points,thres=0.5/max(points), min_dist=min_dist);
            lenght=np.shape(indexes)			# Get the length of the index vector		
            lenght=lenght[0];				# Get the value of the index vector

            for i in range(0,lenght):
                p=indexes[i];
                peaks[p]=points[p];
        
            n=np.arange(0,fil);                            # Vector to the X axes (Number of Samples)
            
            tmp_peaks = np.diff(indexes)
                    
            tmp_interval.append(tmp_peaks)
                    
           
        except:
            print(i)
            tmp_peaks = np.zeros(maxlen)
            tmp_interval.append(tmp_peaks)
                    
     
    padded =pad_sequences(tmp_interval, maxlen=maxlen, dtype='float64', padding='pre', truncating='pre', value=0.0)
        
    for i in range(len(padded)):
        features['interval'].append(padded[i])
        
        
    for i in range(len(features['interval'])):
        features['interval'][i] = np.array(features['interval'][i])
        
    features['interval'] = np.array(features['interval'])
    
    
    
    features['mel1'] = []
    for i in range(len(recordings)) :
        if use_mel :
            mel1 = feature_extract_melspec(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels)[0]
        else :
            mel1 = np.zeros( (1,1) )
            
        features['mel1'].append(mel1)
        
    M, N = features['mel1'][0].shape
    
    if use_mel :
        for i in range(len(features['mel1'])) :
            features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
    
    features['mel1'] = np.array(features['mel1'])

    
  
    features['cqt1'] = []
    for i in range(len(recordings)) :
        if use_cqt :
            mel1 = feature_extract_cqt(recordings[i], samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin)[0]
        else:
            mel1 = np.zeros( (1,1,1) )
            
        features['cqt1'].append(mel1)
        
    M, N,__ = features['cqt1'][0].shape
    
    if use_cqt :
        for i in range(len(features['cqt1'])) :
            features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)   
    
    features['cqt1'] = np.array(features['cqt1'])
    
    
    features['stft1'] = []
    for i in range(len(recordings)) :
        if use_stft :
            mel1 = feature_extract_stft(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                        win_length = win_length)[0]
        else :
            mel1 = np.zeros( (1,1,1) )
        
        features['stft1'].append(mel1)
        
    M, N,__ = features['stft1'][0].shape
    if use_stft :
        for i in range(len(features['stft1'])) :
            features['stft1'][i] = features['stft1'][i].reshape(M,N,1)           
    features['stft1'] = np.array(features['stft1'])
    
#     return features       

    #    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], 
                           features['loc'], features['interval'],features['wav2'],features['mel1'], features['stft1'], features['cqt1']])
    res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], 
                           features['loc'], features['interval'],features['wav2'],features['mel1'], features['stft1'], features['cqt1']])

    # Get classifier probabilities.
    idx1 = res1.argmax(axis=0)[0]
    murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
    outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
#    idx = np.argmax(prob1)

    ## 이부분도 생각 필요.. rule 을 cost를 maximize 하는 기준으로 threshold 탐색 필요할지도..
    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1
    
    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))
    
    return classes, labels, probabilities

In [64]:
model_folder

'lcnn2'

In [65]:
data_folder = test_folder

In [66]:
data_folder

'/home/jk21/Downloads/Data/data/murmur/test'

In [67]:
output_folder ='/home/jk21/Documents/hmd/jk_classifier/out_lcnn2'

In [68]:
allow_failures=True

In [69]:
verbose=1

In [70]:
model = load_challenge_model(model_folder, verbose)

In [71]:
patient_files = find_patient_files(data_folder)

In [72]:
num_patient_files = len(patient_files)

In [73]:
patient_data = load_patient_data(patient_files[0])

In [74]:
patient_data

'33151 4 4000\nAV 33151_AV.hea 33151_AV.wav 33151_AV.tsv\nPV 33151_PV.hea 33151_PV.wav 33151_PV.tsv\nTV 33151_TV.hea 33151_TV.wav 33151_TV.tsv\nMV 33151_MV.hea 33151_MV.wav 33151_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 141.0\n#Weight: 30.9\n#Pregnancy status: False\n#Murmur: Present\n#Murmur locations: MV+TV\n#Most audible location: TV\n#Systolic murmur timing: Holosystolic\n#Systolic murmur shape: Plateau\n#Systolic murmur grading: I/VI\n#Systolic murmur pitch: Low\n#Systolic murmur quality: Harsh\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [75]:
recordings = load_recordings(data_folder, patient_data)

In [76]:
classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose)

100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  9.80it/s]


In [77]:
classes

['Present', 'Unknown', 'Absent', 'Abnormal', 'Normal']

In [78]:
# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose=1):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

In [79]:
test_folder

'/home/jk21/Downloads/Data/data/murmur/test'

In [80]:
output_folder = '/home/jk21/Documents/hmd/jk_classifier/out_lcnn2'

In [81]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 2)

Loading Challenge model...
Running model on Challenge data...
    1/191...


100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  9.44it/s]


    2/191...


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  9.89it/s]


    3/191...


...

    190/191...


100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  9.75it/s]


    191/191...


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  9.01it/s]


Done.


In [82]:
from evaluate_model import *

In [83]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.654,0.545,0.481,0.791,0.601,21526.814

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.706,0.712,0.639,0.639,0.635,13168.934

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.810,0.386,0.766
AUPRC,0.686,0.070,0.878
F-measure,0.571,0.000,0.872
Accuracy,0.474,0.000,0.957

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.706,0.706
AUPRC,0.733,0.692
F-measure,0.642,0.635
Accuracy,0.633,0.645



In [67]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

0.35101086986298435
0.625466137307694
0.6291586638481677
0.3708413376993194


In [68]:
for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")

threshold:  0.01
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.799,0.657,0.234,0.340,0.585,12625.691

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.697,0.694,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.940,0.602,0.855
AUPRC,0.858,0.193,0.921
F-measure,0.380,0.000,0.321
Accuracy,1.000,0.000,0.194

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.697,0.697
AUPRC,0.745,0.642
F-measure,0.678,0.000
Accuracy,1.000,0.000

-------------
threshold:  0.05
#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.799,0.657,0.311,0.450,0.642,11692.456

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.697,0.694,0.339,0.513,0.840,15140.890

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.940,0.602,0.855
AUPRC,0.858,0.193,0.921
F-measure,0.429,0.000,0.503
Accuracy,1.000,0.000,0.345

#Outcome scores (per class)
Classes,Abnormal,Normal
AU